In [ ]:
from bs4 import BeautifulSoup
import pyodbc
import requests
import threading
from itertools import zip_longest
from math import ceil

connection_string = "Driver={SQL Server Native Client 11.0};Server=DESKTOP-R9VI2A2\SQLEXPRESS;Database=fedresurs;Trusted_Connection=yes;"
lock = threading.Lock()

def prepare_date(not_prepared_date):
    resultDate = ""
    try:
        date_mas = not_prepared_date.split(".")
        resultDate = date_mas[2]+"."+date_mas[1]+"."+date_mas[0]
    except:
        resultDate = "1100.10.10"
    return resultDate
      
def send_request(url):
    session = requests.Session()
    response = session.post(url)
    bs = BeautifulSoup(response.text,"html.parser")
    searched_table = bs.find("table", attrs={ "class" : "au"})
    bs = BeautifulSoup(str(searched_table),"html.parser")
    birthday = prepare_date(bs.find("span", attrs={ "id" : "ctl00_cphBody_lblBirthdate"}).text)
    birthplace = bs.find("span", attrs={"id":"ctl00_cphBody_lblBirthplace"}).text
    live_city = bs.find("span", attrs={"id":"ctl00_cphBody_lblAddress"}).text
    return birthday, birthplace, live_city

def search_unprocessed_debtors():
    sqlRequest = """SELECT db.debtor_id, db.debtor_link
    FROM debtorInfo  db LEFT JOIN  debtor_additional_info dai ON   db.debtor_id=dai.debtor_id
    WHERE dai.debtor_id IS NULL"""
    connection = pyodbc.connect(connection_string)
    dbCursor = connection.cursor()
    dbCursor.execute(sqlRequest)
    cursorVocabulary = {}
    for row in dbCursor:
        cursorVocabulary[row.debtor_id] = [row.debtor_link]
    connection.commit()
    connection.close()
    return cursorVocabulary

def search_debtors():
    threads = []
    debtors_list = search_unprocessed_debtors()
    chunks = [debtors_list.items()]*4
    g = list(dict(filter(None,v)) for v in zip_longest(*chunks))
    part_len = ceil(len(g)/4)
    parting_list = [g[part_len*k:part_len*(k+1)] for k in range(4)]
    
    for part in parting_list:
        threads.append(threading.Thread(target=fetch_requests, args=(part,)))
    for thread in threads:
        thread.start()
    for thread in threads:
        thread.join()

def parting(xs, parts):
    part_len = ceil(len(xs)/parts)
    return [xs[part_len*k:part_len*(k+1)] for k in range(parts)]
        
def fetch_requests(list_part):
    parts = parting(list_part,250)
    for part in parts:
        lock.acquire()########################
        resultValues = ""
        for elements in part:
            for key in elements.keys():
                debtor_link = elements[key][0]
                try:
                    birthday,birthplace, live_city = send_request(debtor_link)
                    resultValues += "("+str(key)+",\'"+birthday+"\',\'"+birthplace+"\',\'"+live_city+"\'),"
                    print(str(key))
                except:
                    break
        string_for_insert = resultValues[:-1] #строка для вставки в таблицу addition_debtor_info
        lock.release()##################
        print(string_for_insert)
        insert_into_addition_info(string_for_insert)
        
def insert_into_addition_info(string_for_insert):
    sqlRequest = """INSERT INTO debtor_additional_info (debtor_id, debtor_birthday, debtor_birthplace,debtor_live_city) VALUES"""+string_for_insert
    connection = pyodbc.connect(connection_string)
    dbCursor = connection.cursor()
    dbCursor.execute(sqlRequest)
    connection.commit()
    connection.close()

if __name__=="__main__":
    search_debtors()

In [ ]:
from bs4 import BeautifulSoup
import pyodbc
import requests
import threading
from itertools import zip_longest
from math import ceil
import time
from fake_useragent import UserAgent
import random
from requests.exceptions import InvalidURL

PROXIES =[
{'http':'120.132.52.28'},
{'http':'52.179.231.206'},
{'http':'120.132.52.142'},
{'http':'120.132.53.30'},
{'http':'120.132.52.196'},
{'http':'120.132.52.138'},
{'http':'120.132.52.131'},
{'http':'120.132.52.251'},
{'http':'180.150.191.224'},
{'http':'180.150.191.224'},
{'http':'120.132.52.91'},
]

connection_string = "Driver={SQL Server Native Client 11.0};Server=DESKTOP-R9VI2A2\SQLEXPRESS;Database=fedresurs;Trusted_Connection=yes;"
session = requests.Session()

def prepare_date(not_prepared_date):
    resultDate = ""
    try:
        date_mas = not_prepared_date.split(".")
        resultDate = date_mas[2]+"."+date_mas[1]+"."+date_mas[0]
    except:
        resultDate = "1100.10.10"
    return resultDate

def insert_into_addition_info(string_for_insert):
    sqlRequest = """INSERT INTO debtor_additional_info (debtor_id, debtor_birthday, debtor_birthplace,debtor_live_city) VALUES"""+str(string_for_insert)
    connection = pyodbc.connect(connection_string)
    dbCursor = connection.cursor()
    dbCursor.execute(sqlRequest)
    connection.commit()
    connection.close()

def send_request(url):
    #session = requests.Session()
    ###proxy = random.choice(PROXIES)
    headers = {'user-agent':UserAgent().chrome}####################
    #response = requests.get(url+str(count),proxies=proxy,headers=headers)
    ###response = session.post(url,proxies=proxy, headers=headers)
    response = session.post(url, headers=headers)#####################
    #response = session.post(url)
    bs = BeautifulSoup(response.text,"html.parser")
    searched_table = bs.find("table", attrs={ "class" : "au"})
    bs = BeautifulSoup(str(searched_table),"html.parser")
    birthday = prepare_date(bs.find("span", attrs={ "id" : "ctl00_cphBody_lblBirthdate"}).text)
    birthplace = bs.find("span", attrs={"id":"ctl00_cphBody_lblBirthplace"}).text
    live_city = bs.find("span", attrs={"id":"ctl00_cphBody_lblAddress"}).text
    time.sleep(5)
    return birthday, birthplace, live_city

def search_unprocessed_debtors():
    sqlRequest = """SELECT db.debtor_id, db.debtor_link
    FROM debtorInfo  db LEFT JOIN  debtor_additional_info dai ON   db.debtor_id=dai.debtor_id
    WHERE dai.debtor_id IS NULL"""
    connection = pyodbc.connect(connection_string)
    dbCursor = connection.cursor()
    dbCursor.execute(sqlRequest)
    cursorVocabulary = {}
    for row in dbCursor:
        cursorVocabulary[row.debtor_id] = [row.debtor_link]
    connection.commit()
    connection.close()
    return cursorVocabulary

def search_debtors():
    threads = []
    debtors_list = search_unprocessed_debtors()
    chunks = [debtors_list.items()]*200
    g = list(dict(filter(None,v)) for v in zip_longest(*chunks))
    part_len = ceil(len(g)/200)
    parting_list = [g[part_len*k:part_len*(k+1)] for k in range(200)]
    for part in parting_list:
        resultValues = ""
        for elements in part:
            for key in elements.keys():
                    try:
                        debtor_link = elements[key][0]
                        print(key)
                        birthday,birthplace, live_city = send_request(debtor_link)
                        resultValues += "("+str(key)+",\'"+birthday+"\',\'"+birthplace+"\',\'"+live_city+"\'),"
                    except (InvalidURL):
                        continue
        string_for_insert = resultValues[:-1]+";" #строка для вставки в таблицу addition_debtor_info
        print(string_for_insert)
        insert_into_addition_info(string_for_insert)
        time.sleep(10)

if __name__=="__main__":
    search_debtors()

46901
15072
19261
142718
101303
142677
101322
188030
142754
101342
19223
63155
142659
142665
101262
101276
63226
187976
63232
187983
63253
188010
188017
187998
188009
63100
19309
19313
101344
19331
188050
19336
188051
20084
20093
188054
188055
63184
188086
142834
63220
142856
101437
101446
19392
101467
19447
101505
142782
19455
19456
(19261,'1976.11.11','г.Москва','г. Москва, ул. Маршала Вершинина, д.5 кв. 73'),(142718,'1979.10.01','гор.Борзя Читинской области','607600, Нижегородская область, г.Богородск, ул.Добролюбова, д.1б, кв.7'),(101303,'1961.05.17','гор. Караганда Казахстан','Московская обл., г. Куровское, Новинское шоссе, д.12, кв.2'),(142677,'1980.10.15','гор. Красноярск','г. Красноярск, ул. Новосибирская, д. 29, кв. 32'),(101322,'1956.09.02','гор.Кустанай Кустанайской обл.','350062, г.Краснодар, ул. им.Архитектура Ишунина, д.7/1, корп.2, кв.99'),(188030,'1978.06.21','г. Ступино, Московской области','Московская обл., г. Ступино, ул. Службина, д. 4, кв. 81'),(142754,'1980.06.08'

19440
19452
188225
19471
63418
188243
63420
188246
188411
143084
101662
143109
101530
188255
19483
19498
101570
101572
188274
142986
188277
63451
19521
142995
143003
188306
19552
101597
19559
63475
143015
101605
19628
143079
101651
19592
143022
19593
101612
188347
143030
188362
19608
143040
143041
143047
143060
19622
143070
63517
101650
(19440,'1983.04.10','п.Новатор, Великоустюгского района, Вологодской области','162600, Вологодской обл., г. Череповец, пр. Строителей, д.19, кв.34'),(19452,'1972.10.19','с. Развильное Песчанокопского района Ростовской области','Ростовская область, Песчанокопский район, с. Развильное, ул. Восточная, д.59'),(188225,'1100.10.10','(н/д)','Ярославская обл. Ростовский район пос. Семибратово ул. Ломоносова д.4 кв.10'),(19471,'1984.01.26','гор. Оренбург','Оренбургская область, город Оренбург, ул. Волгоградская, д. 12, кв. 72'),(63418,'1979.12.02','город Бакал Челябинской области','655158, Республика Хакасия, г. Черногорск, ул. Дзержинского, д. 20, кв. 38'),(188

19819
19820
188484
19821
188486
101680
63671
19863
63682
188548
188549
188574
143170
143234
188592
188607
143286
188639
19838
143301
101893
63767
101903
101918
143337
188573
101977
101816
101826
188636
19854
63770
19896
188534
19922
188605
143362
101981
19923
19930
19944
143376
188645
188648
19954
63849
143387
102071
19972
20032
101986
(19819,'1964.09.03','г. Подольск Московская область','Московская область, г. Подольск, ул. Каменная, д. 9/4'),(19820,'1980.03.03','в/ч Ситал-Чай Апшеронского района Азербайджанской ССР','г. Воронеж, ул. Ржевская, д. 9, кв. 162'),(188484,'1965.06.02','деревня Вурманкас-Алманчино Красноармейского района Чувашской АССР','429951, Чувашская Республика, город Новочебоксарск, ул. Комсомольская, д. 14, кв. 519'),(19821,'1965.07.18','пос. Шахровка Омутнинского р-на Кировской обл.','Кировская обл., Омутнинский р-н, пос. Шахровка, ул. Путейская, д. 8'),(188486,'1987.01.09','д. Аксарино Мариинско-Посадский район Чувашской Республики','Чувашская Республика, Мариинско

20082
188813
20075
63864
143555
20090
20091
188879
143569
20099
102103
143590
102120
63959
20148
188798
63997
20159
64004
64018
20177
143483
64031
102181
143533
143535
145020
145023
143589
188896
20108
102137
63963
143673
102174
143480
102195
143566
102221
20219
143528
143537
20266
188925
20284
64103
20293
143608
188940
143622
143628
(20082,'1977.05.15','гор. Малоярославец Калужской обл.','249000, Калужская обл., Малоярославецкий р-н., г. Малоярославец, ул. Гоголя, д. 73'),(188813,'1976.07.21','гор. Свердловск','Свердловская обл., г. Екатеринбург, ул. Парниковая, д. 7, корп. 1 кв. 21'),(20075,'1997.01.31','с. Кадышево Карсунского района Ульяновской обл.','Московская область, Дмитровский район, дер. Бунятино, д. 116'),(63864,'1967.03.19','ГОР. ТЮМЕНЬ','625029, г. Тюмень, ул. Игримская, д. 16, кв. 52'),(143555,'1976.08.18','г.Киров','610007, г.Киров, ул.Тимирязева, д.4, кв.77'),(20090,'1983.08.18','п.Степной Красногвардейского р-на Оренбургской области','398540, Липецкая область,Липецкий

189032
102331
102333
102341
189039
20317
102342
20312
102348
20318
102356
64220
143759
102360
64223
143762
189069
20383
189146
20341
102379
20345
20349
143779
20351
189109
143788
102412
20361
102418
64260
189128
102455
189141
143818
102456
189152
64279
64295
64297
20422
20423
143908
102345
143911
20428
102347
20430
102349
20432
102361
(189032,'1964.10.30','г.Москва','301000,ТО, Заокский р-н, д.Александровка, ул.Магаданская, 38'),(102331,'1969.03.27','г. Новосибирск',' 630010, г. Новосибирск, ул. Рылеева, д.93'),(102333,'1956.04.15','п. Дарасун Карымского р-на Читинской обл','660125, г. Красноярск, ул. Светлогорская, д. 33, кв. 74'),(102341,'1963.12.21','пос. Александровский Прохоровского района Белгородской области','142924, Московская область, Каширский район, деревня Бурцево, ул. Заводская, д.20'),(189039,'1982.01.25','г.Ленинск-Кузнецкий, Кемеровской области','652523, Кемеровская область, г. Ленинск-Кузнецкий, пр. Текстильщиков, 5/1-38'),(20317,'1978.10.03','г. Талды-Курган Казахско

...

103931
22380
103954
103842
146238
146281
103898
146102
103967
103974
190758
146146
103987
22455
22462
22465
146217
104051
146291
146093
65981
65988
22430
22464
104034
104040
104058
104060
190816
103872
103874
74293
22409
22419
66006
104013
104014
22458
66013
66049
22324
66086
146283
190826
103887
103891
146289
66122
190839
146315
103943
(103931,'1970.06.01','г. Иваново','г. Иваново, ул. Светлая, д. 8 кв. 56'),(22380,'1957.07.07','г. Ленинград','г. Архангельск, пр.Троицкий, д.12, кв.36'),(103954,'1978.04.22','д. Сириккасы Аликовского района Чувашской Республики','Чувашская Республика, Аликовский район, д.Сириккасы, ул.Фрунзе, д.39'),(103842,'1963.07.17','сов. им. Пушкина Камышнинского р-на Кустанайского обл.','Орловская область, Корсаковский р-н, дер. Страховка, 15 Б'),(146238,'1988.03.09','г.Кировск Мурманской области','Ярославская область, Некрасовский район, с.Тимохино, д.47'),(146281,'1100.10.10','(н/д)','г. Томск, пр. Кирова, д. 22, кв. 131'),(103898,'1959.05.19','г. Ангарск, Иркут

22601
122998
72227
72271
72286
123071
22619
72312
72326
22634
72331
174138
123107
123109
123112
174061
174063
72358
174073
174081
123153
72396
123168
72410
72436
123185
174146
174161
22766
22767
72256
72480
72267
72268
72270
72279
123226
123228
22806
174199
123233
123234
174206
72310
72322
22637
174051
123252
22659
72444
123378
(22601,'1985.08.04','с. Долгое Вейделевского р-на Белгородской обл.','309738 Белгородская обл., Вейделевский р-н, с. Долгое'),(122998,'1985.11.10','город Кинешма','155827, Ивановская область, Кинешемский район, деревня Пеньки, ул. Колхозная, д. 3'),(72227,'1954.03.08','г. Ростов-на-Дону','Ростовская область, г. Ростов-на-Дону, ул. Зорге, д. 8, кв. 73'),(72271,'1962.11.25','с.Речное Пресногорьковского р-на Кустанайской обл.','г.Челябинск, ул.Колхозная, 25-16'),(72286,'1979.02.17','г. Пенза, Пензенской области','440000, Пензенская область, г. Пенза, ул. Бородина, д. 7, кв. 109'),(123071,'1958.06.13','г. Саратов ','350051, Краснодарский край, г. Краснодар, ул. Рашп

123400
123416
22959
123453
73291
123700
123721
73181
123468
174419
123470
123481
22991
123491
123501
123516
73228
73248
23022
23025
23035
23036
23043
23045
73282
73286
23051
123560
73330
73347
23083
123587
73374
174423
174431
123600
174478
23182
73249
23039
123574
23070
174387
73368
23099
174441
174464
123615
23154
73211
123630
(123400,'1100.10.10','(н/д)','ул. Октябрьская, д2Б, кв 15, Тамбов, Тамбовская область, 392000'),(123416,'1970.01.28','Ново-Батайск Азовского района Ростовской области','347716, Рост. обл, с. Ново-Батайск, ул. Комсомольская, д. 33а'),(22959,'1989.02.14','г.Воронеж','г.Воронеж, ул.Огарева д.3'),(123453,'1955.01.03','город Чита','г. Новосибирск, ул. Дачная, д. 23/5 кв.59'),(73291,'1979.08.17','гор. Красавино Великоустюгского района Вологодской области','162341, Вологодская область, г. Красавино, ул. Кооперативная, д. 13'),(123700,'1987.02.28','Рязань','г. Рязань, ул. Бирюзова, д. 19, кв.51'),(123721,'1978.09.06','г. Санкт-Петербург','Тверская обл., г. Бежецк, ул. Л

73627
123733
73636
73442
124019
23356
123783
174778
174779
123831
23413
73601
174834
23422
123901
174843
174855
23442
73650
174859
174864
123951
73670
73676
73680
23467
174894
123998
73481
123777
123786
174761
174770
23380
174792
123823
174802
174812
123861
23409
174839
23419
174853
23441
174867
23465
123974
123980
73787
23618
124204
(73627,'1965.07.20','г. Железноводск Ставропольского края','357401, Ставропольский край, г. Железноводск, кл. Косякина, д. 32, кв. 24'),(123733,'1979.04.08','с. Лукерьино Исилькулького района Омской области','Омская обл., Исилькульский район, с. Медвежье, ул. Механизаторов, д. 9'),(73636,'1989.03.16','пос. МагниткаКусинского района Челябинской области','г. Челябинск, ул. Энтузиастов, 14,кв.45'),(73442,'1995.09.17','гор. Ярославль',' 150023, Ярославская обл., г. Ярославль, ул. Гагарина, д. 53, кв.'),(124019,'1972.12.29','Челябинск','г.Челябинск, , ул.Доватора, д.24-Б, кв.57'),(23356,'1960.07.12','гор. Душанбе','Красноярский край, г. Зеленогорск, ул. Комсомо

73900
23697
73922
175143
73935
175145
73939
23734
